In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from scipy.stats import randint

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

results = []
print("\ud83d\udccc Model Evaluation:\n")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1
    })

    print(f"\ud83d\udd39 {name}\n{classification_report(y_test, y_pred)}\n")

results_df = pd.DataFrame(results)
print("\ud83d\udcca Performance Summary:")
print(results_df.sort_values(by="F1 Score", ascending=False))

In [ ]:
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='f1_weighted')
grid_search_rf.fit(X_train, y_train)
print("\n✅ Best Parameters (GridSearchCV - RF):", grid_search_rf.best_params_)

In [ ]:
param_dist_xgb = {
    'n_estimators': randint(50, 150),
    'max_depth': randint(3, 10),
    'learning_rate': [0.01, 0.05, 0.1, 0.2]
}

random_search_xgb = RandomizedSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    param_distributions=param_dist_xgb,
    n_iter=10,
    cv=5,
    scoring='f1_weighted',
    random_state=42
)

random_search_xgb.fit(X_train, y_train)
print("✅ Best Parameters (RandomizedSearchCV - XGBoost):", random_search_xgb.best_params_)

In [ ]:
best_model = random_search_xgb.best_estimator_  # or grid_search_rf.best_estimator_
y_pred_best = best_model.predict(X_test)
print("\n🏆 Final Evaluation on Best Model:")
print(classification_report(y_test, y_pred_best))